In [1]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Remote boilerplate/centered_RNN_remote.py'

In [6]:
#Loading XGBRF.v1 model back in (trained on uncentered data) but tested with centered data
import pickle 
from scipy.stats import pearsonr

# Load model in with pickle file 
with open('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/XGBRF/Saved_Models_XGBRF_v1.pkl', 'rb') as file:
    loaded = pickle.load(file)

In [7]:
 # Evaluate model 

 #loaded = pickle.load(file)
 #Loaded object is a list of models. n_models: 16101
 #y_pred.shape: (3187, 16101)
 #model.n_features_in_: None
 #model.n_outputs_: None
 #x_test.shape: (3187, 1198)
 #y_test.shape: (3187, 16101)
 #Multi-output R^2 (uniform_average): 0.7601639389405871


# List of estimators (one per target gene) so build predictions matrix 

if isinstance(loaded, list):
    models = loaded
    print("Loaded object is a list of models. n_models:", len(models))
    y_pred = np.column_stack([m.predict(x_test) for m in models])  # (n_samples, n_genes)
else:
    model = loaded
    print("Loaded object type:", type(model))
    # If single-output model but y_test is multi-column, this will raise -- handled later
    y_pred = model.predict(x_test)

print("y_pred.shape:", y_pred.shape)

# diagnostics (safe access)
def safe_attr(obj, name):
    return getattr(obj, name, None) if not isinstance(obj, list) else None

print("model.n_features_in_:", safe_attr(loaded, "n_features_in_"))
print("model.n_outputs_:", safe_attr(loaded, "n_outputs_"))
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)

# Continue with your R^2 / MSE logic expecting y_pred shape (n_samples, n_targets)
from sklearn.metrics import r2_score, mean_squared_error

# If y_pred is 1D, treat as single-output
if y_pred.ndim == 1 or (y_pred.ndim == 2 and y_pred.shape[1] == 1):
    if y_test.ndim == 2 and y_test.shape[1] > 1:
        raise ValueError(
            "Model predicts a single target but y_test contains multiple targets (genes).\n"
            "Select the trained target column before splitting, e.g.:\n"
            "  target = 'GENE_NAME'\n"
            "  y = gene_expression[target]\n"
            "  then redo train_test_split and evaluation."
        )
    y_true = y_test.ravel()
    print("R^2:", r2_score(y_true, y_pred))
    print("MSE:", mean_squared_error(y_true, y_pred))
    print("PCC:", pearsonr(
                    y_true.detach().flatten().cpu().numpy(),
                    y_pred.detach().flatten().cpu().numpy()))
else:
    print("Multi-output R^2 (uniform_average):", r2_score(y_test, y_pred, multioutput='uniform_average'))
    
    pcc_per_target = np.array([
    pearsonr(y_test[:, i], y_pred[:, i])[0]
    for i in range(y_test.shape[1])
        ])
    mean_pcc = np.nanmean(pcc_per_target)
    print("Multi-output PCC (mean over targets):", mean_pcc)
    
    
first = models[0]
print("first estimator type:", type(first))
print("n_features_in_:", getattr(first, "n_features_in_", None))
print("example feature_importances_ (first 10):", getattr(first, "feature_importances_", None)[:10])


Loaded object is a list of models. n_models: 16101
y_pred.shape: (3187, 16101)
model.n_features_in_: None
model.n_outputs_: None
x_test.shape: (3187, 1198)
y_test.shape: (3187, 16101)
Multi-output R^2 (uniform_average): 0.7601639389405871
Multi-output PCC (mean over targets): 0.8680410938624922
first estimator type: <class 'xgboost.sklearn.XGBRFRegressor'>
n_features_in_: 1198
example feature_importances_ (first 10): [0.02488494 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
